# 第10章 节省空间

## 10.1 示例问题

问题描述：有一个在地理数据库中存储邻居的系统，一共两千个邻居，编号范围为0\~1999，每个邻居在地图中用一个点来描述。该系统允许用户通过触摸输入板的方式访问其中的任意一个点。

问题分析：使用$(x, y)$表示邻居在地图上的点，考虑用$200 \times 200$的点标识符数组表示地图的模块，点标识符是0\~1999的整数，如果该位置没有点，点标识符置为-1。

数据结构表示：

- 原始方案：使用链表表示给定列中的活跃元素。使用一个具有200个指针以及2000条记录的数组，每个记录都有一个整数和两个指针，每条记录需要占用12个字节，2000元的数组需要占用24800个字节。
> colhead &emsp; row,  next  
> &emsp; 0 &emsp;&emsp;$\rightarrow$ [2, 17] $\rightarrow$ [5, 538] $\rightarrow$ [126, 1053]  
> &emsp; 1 &emsp;&emsp;$\rightarrow$ [1, 98] $\rightarrow$ [138, 15]  
> &emsp; 2 &emsp;&emsp;$\rightarrow$ ...

- 改进方案：使用一个201元的数组，以及两个2000元的并行数组表示。用16位整数实现，可节约70KB空间。空间压缩到了6400个字节。
> pointnum [17, 538, 1053, 98, 15, 1800, ..., 437, 832]  
> row &emsp;&emsp;&nbsp;&nbsp;[2, 5, 126, 1, 138, 11, ..., 111, 67]  
> firstincol &nbsp;&nbsp;[0, 3, 5, 5, ... , 1998, 2000]  
> &emsp;&emsp;&emsp;&emsp;&nbsp; 0 &nbsp; 1 &nbsp;2&nbsp; 3 &emsp;&emsp;&nbsp;199&emsp;&nbsp;200

## 10.2 数据空间技术

- 不存储，重新计算：如果在需要某一给定对象的任何时候，都对其进行重新计算而不保存，保存该对象所需的空间就可以急剧下降。
- 稀疏数据结构：如果使用关键字作为索引存储到表中，就不需要存储关键字本身，只需要存储其相关的属性。
- 数据压缩：通过压缩的方式对对象进行编码，以减少存储空间。
- 分配策略：动态分配通过在需要时对记录进行分配，降低资源的浪费。
- 垃圾回收：对废弃的存储空间进行回收再利用。

## 10.3 代码空间技术

代码转换：
- 第1个版本：`for i = [17, 43] set(i, 68)`
- 第2个版本：`hor(17, 43, 68)`
- 第3个版本：`h 17 43 68`

代码空间技术：
- 函数定义：通过用函数代替代码中的常见模式，可以简化程序，相应地减少了程序的空间需求。
- 解释程序：在图形程序中，用4字节的解释程序命令替换了一长行的程序文本。
- 翻译成机器语言：将源语言转换成机器语言，

## 10.4 巨大的节省（边栏）

**背景：** 

20世纪80年代早期，Ken Thompson构建一个两阶段程序，用于解决给定条件下国际象棋的残局问题。通过从所有可能的“将死”状态向前回溯，计算所有可能的走法的距离。

**空间开销：**

将棋盘的编码作为官架子，对存储棋盘信息的磁盘文件进行索引，文件中每条记录包含12位，棋盘上有64个格子，其中五个固定棋子位置可以编码为0\~63的5个整数，可得到关键字有30位，数据库中的表有$2^{30}$个12位的记录，已经超过了当时可用的磁盘容量。

**空间优化：**

将棋盘进行对角线划分，对于任意的棋盘，至多连续三次镜像就可以摆成指定形式，可以使磁盘文件的大小减小到$10 \times 64^4$或$10 \times 2^{24}$个12位的记录，对于指定的两个王，只有454种合法的棋盘位置。

**优化结果：**

将数据库缩小到$454 \times 64^3$或大约12100万条12位的记录，可以保存在一张专用的磁盘中。

**总结：**

Thompson利用数据结构的对称性，使所需磁盘空间减少为原来的八分之一。节省空间的同时也减少了程序的运行时间：通过减少在残局程序中需要分析的位置数，将学习阶段的时间从几个月减少到了几周时间。